## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [4]:
import mysql.connector 

import mysql.connector

cnx = mysql.connector.connect(
    host = "localhost",
    user = "nortonacosta",
    password = "12345"
)

print(cnx)


Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [5]:
cursor = cnx.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [6]:
cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;") 


In [7]:
cursor.execute("SHOW DATABASES;")

for db in cursor:
    print(db)


('dbprodutos',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [8]:
import pandas as pd
df_livros = pd.read_csv("/home/nortonacosta/EngDados/pipeline-python-mongo-mysql/data/tabela_livros.csv")
df_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,666115bd34467c814f135261,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,666115bd34467c814f135262,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,666115bd34467c814f135274,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,666115bd34467c814f135282,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,666115bd34467c814f135284,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


In [9]:
df_livros.columns #visualizar nome das colunas

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [10]:
df_livros.shape #retorna quantidade de Linhas e Colunas

(742, 13)

In [12]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               
               PRIMARY KEY (id));
""")


**Selecionando a base de dados para verificar a tabela criada**

In [13]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
  print(tb)



('tb_livros',)


## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [14]:
for i, row in df_livros.iterrows(): #Passa por todas as linhas
    print(i)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
for i, row in df_livros.iterrows(): #passa pelas informações
    print(row)


_id                       666115bd34467c814f135261
Produto                        Modelagem preditiva
Categoria do Produto                        livros
Preço                                        92.45
Frete                                     5.609697
Data da Compra                          2020-01-01
Vendedor                              Thiago Silva
Local da compra                                 BA
Avaliação da compra                              1
Tipo de pagamento                   cartao_credito
Quantidade de parcelas                           3
Latitude                                    -13.29
Longitude                                   -41.71
Name: 0, dtype: object
_id                       666115bd34467c814f135262
Produto                   Iniciando em programação
Categoria do Produto                        livros
Preço                                        43.84
Frete                                          0.0
Data da Compra                          2020-01-01
Vendedor

In [16]:
for i, row in df_livros.iterrows(): #transforma em uma tupla
    print(tuple(row))


('666115bd34467c814f135261', 'Modelagem preditiva', 'livros', 92.45, 5.6096965236, '2020-01-01', 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('666115bd34467c814f135262', 'Iniciando em programação', 'livros', 43.84, 0.0, '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('666115bd34467c814f135274', 'Iniciando em programação', 'livros', 63.25, 3.894137208, '2022-01-01', 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('666115bd34467c814f135282', 'Ciência de dados com python', 'livros', 86.13, 5.2731755368, '2021-02-01', 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('666115bd34467c814f135284', 'Ciência de dados com python', 'livros', 72.75, 1.458157705, '2021-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('666115bd34467c814f135287', 'Iniciando em programação', 'livros', 61.73, 0.0, '2021-02-01', 'Beatriz Moraes', 'SP', 4, 'cartao_credito', 1, -22.19, -48.79)
('666115bd34467c814f1352

In [17]:
lista_dados = [tuple(row) for i, row in df_livros.iterrows()] #salvando tuplas em uma lista com list comprehension
lista_dados


[('666115bd34467c814f135261',
  'Modelagem preditiva',
  'livros',
  92.45,
  5.6096965236,
  '2020-01-01',
  'Thiago Silva',
  'BA',
  1,
  'cartao_credito',
  3,
  -13.29,
  -41.71),
 ('666115bd34467c814f135262',
  'Iniciando em programação',
  'livros',
  43.84,
  0.0,
  '2020-01-01',
  'Mariana Ferreira',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('666115bd34467c814f135274',
  'Iniciando em programação',
  'livros',
  63.25,
  3.894137208,
  '2022-01-01',
  'Juliana Costa',
  'RJ',
  5,
  'cartao_credito',
  4,
  -22.25,
  -42.66),
 ('666115bd34467c814f135282',
  'Ciência de dados com python',
  'livros',
  86.13,
  5.2731755368,
  '2021-02-01',
  'Camila Ribeiro',
  'RJ',
  4,
  'cartao_credito',
  3,
  -22.25,
  -42.66),
 ('666115bd34467c814f135284',
  'Ciência de dados com python',
  'livros',
  72.75,
  1.458157705,
  '2021-02-01',
  'Beatriz Moraes',
  'PR',
  4,
  'cartao_credito',
  2,
  -24.89,
  -51.55),
 ('666115bd34467c814f135287',
  'Iniciando em progr

In [18]:
#Adicionando os dados na tabela
sql = "INSERT INTO dbprodutos.tb_livros VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

cursor.executemany(sql, lista_dados)
cnx.commit()


In [19]:
print(cursor.rowcount, "dados foram inseridos.")


742 dados foram inseridos.


## Visualizando os dados inseridos

In [26]:
cursor.execute("SELECT * FROM dbprodutos.tb_livros;")

for row in cursor:
    print(row)


('666115bd34467c814f135261', 'Modelagem preditiva', 'livros', 92.45, 5.61, datetime.date(2020, 1, 1), 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('666115bd34467c814f135262', 'Iniciando em programação', 'livros', 43.84, 0.0, datetime.date(2020, 1, 1), 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('666115bd34467c814f135274', 'Iniciando em programação', 'livros', 63.25, 3.89, datetime.date(2022, 1, 1), 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('666115bd34467c814f135282', 'Ciência de dados com python', 'livros', 86.13, 5.27, datetime.date(2021, 2, 1), 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('666115bd34467c814f135284', 'Ciência de dados com python', 'livros', 72.75, 1.46, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('666115bd34467c814f135287', 'Iniciando em programação', 'livros', 61.73, 0.0, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'SP', 4, 'cartao_credi

In [27]:
cursor.close()

True

In [28]:
cnx.close()

## Inserindo os dados do csv na tabelas: filtrando produtos vendidos a partir de 2021

In [20]:
#Criando tabela
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_produtos_2021_em_diante(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),

               PRIMARY KEY (id));
""")


In [21]:
#leitura do arquivo
import pandas as pd
df_produtos = pd.read_csv("/home/nortonacosta/EngDados/pipeline-python-mongo-mysql/data/tabela_2021_em_diante.csv")
df_produtos.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,666115bd34467c814f13526a,Xadrez de madeira,brinquedos,25.23,0.000000,2021-01-01,Thiago Silva,BA,5,cartao_credito,2,-13.29,-41.71
1,666115bd34467c814f13526b,Impressora,eletronicos,322.04,14.732100,2021-01-01,João Souza,SP,3,cartao_credito,1,-22.19,-48.79
2,666115bd34467c814f13526c,Mesa de centro,moveis,282.22,12.611805,2021-01-01,João Souza,SP,3,boleto,1,-22.19,-48.79
3,666115bd34467c814f13526d,Tablet ABXY,eletronicos,1100.42,68.451348,2021-01-01,Beatriz Moraes,SP,1,cupom,1,-22.19,-48.79
4,666115bd34467c814f13526e,Fogão,eletrodomesticos,791.81,42.444626,2021-01-01,Juliana Costa,SP,4,boleto,1,-22.19,-48.79


In [22]:
#quantidade de linhas que ele possui
df_produtos.shape

(6574, 13)

In [23]:
#Salvando dados na tabela
lista_dados = [tuple(row) for i, row in df_produtos.iterrows()]
sql = "INSERT INTO dbprodutos.tb_produtos_2021_em_diante VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

cursor.executemany(sql, lista_dados)
cnx.commit()


In [25]:
#Informando quantidade de dados insridos
print(cursor.rowcount, "dados foram inseridos.")

6574 dados foram inseridos.
